In [15]:
import pandas as pd
import numpy as np
import joblib

# Load data
df = pd.read_csv("../data/processed/2023_pace_laps_with_delta_phase.csv")

# Load trained model
model = joblib.load(
    "../models/lap_delta_random_forest_prelap_phase.pkl"
)

FEATURES = [
    "TyreLife",
    "TyreLifeSquared",
    "LapInStint",
    "LapNumber",
    "Stint",
    "Compound_encoded",
    "Driver_encoded",
    "Team_encoded",
    "FuelProxy",
    "RaceProgress",
]

race = "Bahrain Grand Prix"
pit_loss_seconds = 22.0

race_df = df[df["RaceName"] == race].copy()


In [16]:
def predict_lap_delta(row):
    X = pd.DataFrame([row[FEATURES]], columns=FEATURES)
    return model.predict(X)[0]


def simulate_race_driver(race_df, driver, pit_lap):
    driver_df = race_df[race_df["Driver"] == driver].sort_values("LapNumber")
    max_lap = int(driver_df["LapNumber"].max())

    total_delta = 0.0

    for _, row in driver_df.iterrows():
        if row["LapNumber"] < pit_lap:
            total_delta += predict_lap_delta(row)

        elif row["LapNumber"] == pit_lap:
            total_delta += predict_lap_delta(row) + pit_loss_seconds

        else:
            new_row = row.copy()
            new_row["TyreLife"] = row["LapNumber"] - pit_lap + 1
            new_row["TyreLifeSquared"] = new_row["TyreLife"] ** 2
            new_row["LapInStint"] = new_row["TyreLife"]
            total_delta += predict_lap_delta(new_row)

    return total_delta


In [17]:
results = []

drivers = ["VER"]

for driver in drivers:
    driver_df = race_df[race_df["Driver"] == driver]
    max_lap = int(driver_df["LapNumber"].max())

    best_delta = float("inf")
    best_pit_lap = None

    for pit_lap in range(10, max_lap - 10):
        delta = simulate_race_driver(race_df, driver, pit_lap)

        if delta < best_delta:
            best_delta = delta
            best_pit_lap = pit_lap

    results.append({
        "Driver": driver,
        "OptimalPitLap": best_pit_lap,
        "TotalLapDelta": best_delta
    })

results_df = pd.DataFrame(results).sort_values("OptimalPitLap")
results_df


,Driver,OptimalPitLap,TotalLapDelta
0,VER,42,-62.423254
